<a href="https://colab.research.google.com/github/benardonyango/FloodMapping_Google_Earth_Enginine_PythonAPI/blob/master/FloodMapping_Senitnel1_GEE_PyhtonAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:

# ---Things to note---

#This is a python script that ustilises Google Earth Python API and Sentinel1 images to determine flood extent. 
#It determines the flood extent by calculating the difference between two periods i.e just before floods and after/during floods. The assumption is that the only things that has changed between the two periods is the added water. 
#Therefore it is important to choose a period just beofre the floods
#Kindly change the Area of Interest and the dates in case you want to run this script for a different area and periods
#Run this script in a jupyter notebook environment Google Colab


# ---- end ----

#Install Google Earth Engine Python API if not installed
#!pip install earthengine-api
#Install folium if not installed
#!pip install folium. Folium enables map visualization
#install geehydro if not installed. It allows for the overlay of data on the basemap
#!pip install geehydro

#import all the installed packages
import ee
import folium
import geehydro

In [0]:
## Trigger the authentication of the API flow. You only need to do this once. A link will be generated. Open that link in a new browser and allow access of Earth Engine API to your google account. Ensure you have a GEE account
ee.Authenticate()

# Initialize the library.
ee.Initialize()

In [0]:
# Load the Sentinel-1 ImageCollection.


#this Area of interest is in the Southern part of Somalia. around River Juba and Shabelle. The area is notorious for flooding during the rainy seasons

roi = ee.Geometry.Rectangle(42.106634,-0.893096, 45.732123,5.907894);

# Loading sentinel1 images and filtering based on date, Polarization type, bounds Metadata and Instrument Mode. In this case vertical transmit and vertical receive is selected

collectionVV = ee.ImageCollection('COPERNICUS/S1_GRD') \
    .filter(ee.Filter.eq('instrumentMode', 'IW')) \
    .filterDate('2020-02-01','2020-05-06') \
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
    .filterBounds(roi) \
    .filterMetadata('resolution_meters', 'equals' , 10) \
    .select('VV');



# Loading sentinel1 images and filtering based on date, Polarization type, bounds Metadata and Instrument Mode. In this case vertical transmit and horizontal receive is selected

collectionVH = ee.ImageCollection('COPERNICUS/S1_GRD') \
    .filter(ee.Filter.eq('instrumentMode', 'IW')) \
    .filterDate('2020-02-01','2020-05-06') \
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))\
    .filterMetadata('resolution_meters', 'equals' , 10) \
    .filterBounds(roi) \
    .select('VH');



# Load Sentinel-1 C-band SAR Ground Range collection (log scale, VVVH, descending)
beforeVVVH = ee.ImageCollection('COPERNICUS/S1_GRD') \
    .filterDate('2020-02-15','2020-03-10') \
    .filter(ee.Filter.eq('instrumentMode', 'IW')) \
    .filterMetadata('transmitterReceiverPolarisation','equals',['VV','VH']) \
    .filterBounds(roi) \
    .select('VV','VH') \
    .mean();
beforeVVVH = beforeVVVH.addBands(beforeVVVH.select('VV').divide(beforeVVVH.select('VH'))).rename(['VV','VH','Ratio']);

#Load Sentinel-1 C-band SAR Ground Range collection (log scale, VVVH, descending)

afterVVVH = ee.ImageCollection('COPERNICUS/S1_GRD') \
    .filterDate('2020-03-20','2020-05-06') \
    .filter(ee.Filter.eq('instrumentMode', 'IW')) \
    .filterMetadata('transmitterReceiverPolarisation','equals',['VV','VH']) \
    .filterBounds(roi) \
    .select('VV','VH') \
    .mean();

afterVVVH = afterVVVH.addBands(afterVVVH.select('VV').divide(afterVVVH.select('VH'))).rename(['VV','VH','Ratio']);



#Filter by date. Filtering the data just before and during / after the floods. The idea is to get a difference between the two. The resulting layer will to some extent be assumed to be flooded area.
beforeVV = collectionVV.filterDate('2020-02-15','2020-03-10').mosaic();
afterVV = collectionVV.filterDate('2020-05-01','2020-05-06').mosaic();
beforeVH = collectionVH.filterDate('2020-02-15','2020-03-10').mosaic();
afterVH = collectionVH.filterDate('2020-05-01','2020-05-06').mosaic();


#Apply filter to reduce speckle
SMOOTHING_RADIUS = 50;
beforeVV_filtered = beforeVV.focal_mean(SMOOTHING_RADIUS, 'circle', 'meters');
beforeVH_filtered = beforeVH.focal_mean(SMOOTHING_RADIUS, 'circle', 'meters');
afterVV_filtered = afterVV.focal_mean(SMOOTHING_RADIUS, 'circle', 'meters');
afterVH_filtered = afterVH.focal_mean(SMOOTHING_RADIUS, 'circle', 'meters');

In [0]:
# Adding the centre of the map to a point in the Area of interest

Map = folium.Map(location=[44.171549,2.928640], zoom_start=8)
Map.setCenter(44.171549,2.928640)


In [0]:
#Calculate difference between before and after

differenceVH = afterVH_filtered.divide(beforeVH_filtered)

#Apply Threshold

DIFF_UPPER_THRESHOLD = 1.35
differenceVH_thresholded = differenceVH.gt(DIFF_UPPER_THRESHOLD)

#Adding datasets to the map
#important ones
Map.addLayer(differenceVH_thresholded.updateMask(differenceVH_thresholded.clip(roi)),{'palette':"0000FF"},'flooded areas - blue',1)
Map.addLayer(roi, {'color': 'red'}, 'Area of Interest')


#Other datasets. Uncomment the lines below to visualize the data in the map

Map.addLayer(beforeVV.clip(roi), {'min':-25,'max':0}, 'Before flood VV', 0)
#Map.addLayer(afterVV.clip(roi), {'min':-25,'max':0}, 'After flood VV', 0)
#Map.addLayer(beforeVH.clip(roi), {'min':-30,'max':0}, 'Before flood VH', 0)
#Map.addLayer(afterVH.clip(roi), {'min':-30,'max':0}, 'After flood VH', 0)
#Map.addLayer(beforeVH.addBands(afterVH).addBands(beforeVH).clip(roi), {'min': -30, 'max': -8},'BVH/AVH/BVH composite', 1)
#Map.addLayer(beforeVH.addBands(afterVV).addBands(afterVH).clip(roi), {'min': -30, 'max': -8},'BVH/AVV/AVH composite', 1)
#Map.addLayer(beforeVV_filtered.clip(roi), {'min':-25,'max':0}, 'Before Flood VV Filtered',0)
#Map.addLayer(beforeVH_filtered.clip(roi), {'min':-30,'max':0}, 'Before Flood VH Filtered',0)
#Map.addLayer(afterVV_filtered.clip(roi), {'min':-25,'max':0}, 'After Flood VV Filtered',0)
#Map.addLayer(afterVH_filtered.clip(roi), {'min':-30,'max':0}, 'After Flood VH Filtered',0)
#Map.addLayer(differenceVH.clip(roi), {'min': 0,'max':2},'difference VH filtered', 0)


In [144]:
#Adding controls to the map
Map.setControlVisibility(layerControl=True, fullscreenControl=True, latLngPopup=True)
#Displaying the map
Map